In [77]:
import glob
import numpy as np
import pandas as pd
import xgboost as xgb
import category_encoders as ce
from sklearn.model_selection import train_test_split


files = glob.glob('csv/train/*.csv')
data_list = []
for file in files:
    data_list.append(pd.read_csv(file, index_col=0))
df = pd.concat(data_list)

/tmp/ipykernel_631/1449228024.py:12: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_list.append(pd.read_csv(file, index_col=0))
/tmp/ipykernel_631/1449228024.py:12: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_list.append(pd.read_csv(file, index_col=0))


In [7]:
df.head()

,種類,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,...,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log
ID,,,,,,,,,,,,,,,,,,,,,
1060685,中古マンション等,NaN,1108,北海道,札幌市厚別区,大谷地東,大谷地,8,３ＬＤＫ,80,...,NaN,NaN,NaN,準工業地域,60.0,200.0,2009年第４四半期,未改装,NaN,7.079181
1005580,中古マンション等,NaN,1101,北海道,札幌市中央区,南９条西,中島公園,5,１ＤＫ,30,...,NaN,NaN,NaN,近隣商業地域,80.0,300.0,2018年第３四半期,未改装,NaN,6.755875
1001363,中古マンション等,NaN,1101,北海道,札幌市中央区,北３条西,西１１丁目,11,３ＬＤＫ,65,...,NaN,NaN,NaN,近隣商業地域,80.0,300.0,2010年第２四半期,未改装,NaN,7.204120
1052374,中古マンション等,NaN,1108,北海道,札幌市厚別区,厚別中央２条,ひばりが丘(北海道),5,４ＬＤＫ,90,...,NaN,NaN,NaN,商業地域,80.0,400.0,2015年第３四半期,未改装,NaN,7.591065
1059107,中古マンション等,NaN,1108,北海道,札幌市厚別区,厚別東４条,新さっぽろ,12,４ＬＤＫ,80,...,NaN,NaN,NaN,準工業地域,60.0,200.0,2012年第４四半期,未改装,NaN,6.875061


In [74]:
"""
アルゴリズムの確認用

"""

# #全データNaNのカラム抽出
# nonnull_list = []
# for col in df.columns:
#     nonnull = df[col].count()
#     if nonnull == 0:
#         nonnull_list.append(col)

# #nullカラムをドロップ
# df = df.drop(nonnull_list,axis=1)

# #必要ないカラムのドロップ
# # df = df.drop("市区町村名", axis=1)
# # df = df.drop("種類", axis=1)

# dis = {
#         "30分?60分":45,
#         "1H?1H30":75,
#         "2H?":120,
#         "1H30?2H":105
#     }

# df["最寄駅：距離（分）"] = df["最寄駅：距離（分）"].replace(dis).astype(float)
# df["面積（㎡）"] = df["面積（㎡）"].replace("2000㎡以上", 2000).astype(float)


# #建築年の数値への置き換え
# y_list = {}
# for i in df["建築年"].value_counts().keys():
#         if "平成" in i:
#             num = float(i.split("平成")[1].split("年")[0])
#             year = 33 - num
#         if "令和" in i:
#             num = float(i.split("令和")[1].split("年")[0])
#             year = 3 - num
#         if "昭和" in i:
#             num = float(i.split("昭和")[1].split("年")[0])
#             year = 96 - num
#         y_list[i] = year
# y_list["戦前"] = 76
# df["建築年"] = df["建築年"].replace(y_list)

# year = {
#         "年第１四半期": ".25",
#         "年第２四半期": ".50",
#         "年第３四半期": ".75",
#         "年第４四半期": ".99"
#     }
# year_list = {}
# for i in df["取引時点"].value_counts().keys():
#         for k, j in year.items():
#             if k in i:
#                 year_rep = i.replace(k, j)
#         year_list[i] = year_rep
# year_list[i] = year_rep
# df["取引時点"] = df["取引時点"].replace(year_list).astype(float)

In [78]:
def data_pre(df):
    nonnull_list = []
    for col in df.columns:
        nonnull = df[col].count()
        if nonnull == 0:
            nonnull_list.append(col)
    df = df.drop(nonnull_list, axis=1)

    df = df.drop("市区町村名", axis=1)

    df = df.drop("種類", axis=1)

    dis = {
        "30分?60分":45,
        "1H?1H30":75,
        "2H?":120,
        "1H30?2H":105
    }
    df["最寄駅：距離（分）"] = df["最寄駅：距離（分）"].replace(dis).astype(float)

    df["面積（㎡）"] = df["面積（㎡）"].replace("2000㎡以上", 2000).astype(float)


    y_list = {}
    for i in df["建築年"].value_counts().keys():
        if "平成" in i:
            num = float(i.split("平成")[1].split("年")[0])
            year = 33 - num
        if "令和" in i:
            num = float(i.split("令和")[1].split("年")[0])
            year = 3 - num
        if "昭和" in i:
            num = float(i.split("昭和")[1].split("年")[0])
            year = 96 - num
        y_list[i] = year
    y_list["戦前"] = 76
    df["建築年"] = df["建築年"].replace(y_list)

    year = {
        "年第１四半期": ".25",
        "年第２四半期": ".50",
        "年第３四半期": ".75",
        "年第４四半期": ".99"
    }
    year_list = {}
    for i in df["取引時点"].value_counts().keys():
        for k, j in year.items():
            if k in i:
                year_rep = i.replace(k, j)
        year_list[i] = year_rep
    df["取引時点"] = df["取引時点"].replace(year_list).astype(float)
    
    cols = ["都道府県名", "地区名", "最寄駅：名称", "間取り", "建物の構造", "用途", "今後の利用目的", "都市計画", "改装", "取引の事情等"]
    ce_df = ce.OrdinalEncoder(cols=cols, handle_unknown='impute')
    df = ce_df.fit_transform(df)
    
    return df

In [79]:
df = data_pre(df)

In [81]:
#質的変数をカテゴリ変数に変換
cols = ["都道府県名", "地区名", "最寄駅：名称", "間取り", "建物の構造", "用途", "今後の利用目的", "都市計画", "改装", "取引の事情等"]
ce_df = ce.OrdinalEncoder(cols=cols, handle_unknown='impute')
df = ce_df.fit_transform(df)

In [82]:
df.head()

,市区町村コード,都道府県名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log
ID,,,,,,,,,,,,,,,,,,
1060685,1108,1,1,1,8.0,1,80.0,26.0,1,1,1,1,60.0,200.0,2009.99,1,1,7.079181
1005580,1101,1,2,2,5.0,2,30.0,39.0,1,2,2,2,80.0,300.0,2018.75,1,1,6.755875
1001363,1101,1,3,3,11.0,1,65.0,30.0,1,1,1,2,80.0,300.0,2010.50,1,1,7.204120
1052374,1108,1,4,4,5.0,3,90.0,6.0,2,1,2,3,80.0,400.0,2015.75,1,1,7.591065
1059107,1108,1,5,5,12.0,3,80.0,39.0,2,1,1,1,60.0,200.0,2012.99,1,1,6.875061


In [86]:
df_train, df_val = train_test_split(df, test_size=0.2)

col = '取引価格（総額）_log'
train_y = df_train[col]
train_x = df_train.drop(col, axis=1)

val_y = df_val[col]
val_x = df_val.drop(col, axis=1)

train_data = xgb.DMatrix(train_x, label=train_y)
eval_data = xgb.DMatrix(val_x, label=val_y)

xgb_params = {
    "objective": "reg:squarederror",
    'eval_metric': "mae"
    }

evals = [(train_data, 'train'), (eval_data, 'eval')]

gbm = xgb.train(xgb_params, train_data, num_boost_round=100, early_stopping_rounds=10, evals=evals,)

[0]	train-mae:0.21882	eval-mae:0.21875
[1]	train-mae:0.18329	eval-mae:0.18346
[2]	train-mae:0.16197	eval-mae:0.16233
[3]	train-mae:0.14867	eval-mae:0.14921
[4]	train-mae:0.13954	eval-mae:0.14022
[5]	train-mae:0.13282	eval-mae:0.13362
[6]	train-mae:0.12849	eval-mae:0.12943
[7]	train-mae:0.12487	eval-mae:0.12588
[8]	train-mae:0.12232	eval-mae:0.12333
[9]	train-mae:0.12024	eval-mae:0.12126
[10]	train-mae:0.11777	eval-mae:0.11883
[11]	train-mae:0.11656	eval-mae:0.11765
[12]	train-mae:0.11517	eval-mae:0.11627
[13]	train-mae:0.11431	eval-mae:0.11547
[14]	train-mae:0.11340	eval-mae:0.11458
[15]	train-mae:0.11248	eval-mae:0.11374
[16]	train-mae:0.11171	eval-mae:0.11299
[17]	train-mae:0.11132	eval-mae:0.11263
[18]	train-mae:0.11072	eval-mae:0.11203
[19]	train-mae:0.11019	eval-mae:0.11153
[20]	train-mae:0.10956	eval-mae:0.11094
[21]	train-mae:0.10907	eval-mae:0.11047
[22]	train-mae:0.10833	eval-mae:0.10974
[23]	train-mae:0.10792	eval-mae:0.10936
[24]	train-mae:0.10696	eval-mae:0.10843
[25]	train

## 分類で実装

In [26]:
import xgboost as xgb
import pandas as pd
import numpy as np
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split

# Kerasに付属の手書き数字画像データをダウンロード
np.random.seed(0)
(X_train_base, labels_train_base), (X_test, labels_test) = mnist.load_data()


# Training set を学習データ（X_train, labels_train）と検証データ（X_validation, labels_validation）に8:2で分割する
X_train, X_validation, labels_train, labels_validation = train_test_split(X_train_base, labels_train_base, test_size = 0.2)


# 各画像は行列なので1次元に変換→X_train,X_validation,X_testを上書き
X_train = X_train.reshape(-1,784)
X_validation = X_validation.reshape(-1,784)
X_test = X_test.reshape(-1,784)

#正規化
X_train = X_train.astype('float32')
X_validation = X_validation.astype('float32')
X_test = X_test.astype('float32')

#ピクセル値が0-255なので255で割って標準化
X_train /= 255
X_validation /= 255
X_test /= 255

# 訓練・テストデータの設定
train_data = xgb.DMatrix(X_train, label=labels_train)
eval_data = xgb.DMatrix(X_validation, label=labels_validation)
X_data = xgb.DMatrix(X_test, label=labels_test)

#経過時間計測
import time
start = time.time()

xgb_params = {
    # 多値分類問題
    'objective':'multi:softmax',
    # クラス数
    'num_class': 10,
    # 学習用の指標 (Multiclass logloss)
    'eval_metric': 'mlogloss',
    }
evals = [(train_data, 'train'), (eval_data, 'eval')]

gbm = xgb.train(
    xgb_params,
    train_data,
    num_boost_round=100,
    early_stopping_rounds=10,
    evals=evals,
    )

preds = gbm.predict(X_data)
from sklearn.metrics import accuracy_score
print('accuracy_score:{}'.format(accuracy_score(labels_test, preds)))

#経過時間
print('elapsed_timetime:{}'.format(time.time()-start))

[0]	train-mlogloss:1.36144	eval-mlogloss:1.37976
[1]	train-mlogloss:1.02846	eval-mlogloss:1.05534
[2]	train-mlogloss:0.80982	eval-mlogloss:0.84437
[3]	train-mlogloss:0.65517	eval-mlogloss:0.69465
[4]	train-mlogloss:0.54093	eval-mlogloss:0.58554
[5]	train-mlogloss:0.45045	eval-mlogloss:0.50003
[6]	train-mlogloss:0.37916	eval-mlogloss:0.43260
[7]	train-mlogloss:0.32197	eval-mlogloss:0.37788
[8]	train-mlogloss:0.27675	eval-mlogloss:0.33492
[9]	train-mlogloss:0.24008	eval-mlogloss:0.30108
[10]	train-mlogloss:0.21141	eval-mlogloss:0.27424
[11]	train-mlogloss:0.18689	eval-mlogloss:0.25121
[12]	train-mlogloss:0.16618	eval-mlogloss:0.23193
[13]	train-mlogloss:0.14792	eval-mlogloss:0.21512
[14]	train-mlogloss:0.13354	eval-mlogloss:0.20234
[15]	train-mlogloss:0.12111	eval-mlogloss:0.19093
[16]	train-mlogloss:0.10954	eval-mlogloss:0.18034
[17]	train-mlogloss:0.10040	eval-mlogloss:0.17192
[18]	train-mlogloss:0.09171	eval-mlogloss:0.16383
[19]	train-mlogloss:0.08409	eval-mlogloss:0.15700
[20]	train